In [35]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

In [36]:
policy = pd.read_csv('./data/processed/policy_clean.csv')

In [37]:
policy.columns

Index(['policy_id', 'policy_start_date', 'policy_end_date', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'premium', 'exposure',
       'earned_premium'],
      dtype='object')

In [39]:
# Merge frequency and severity predictions into pricing table
freq_pred = pd.read_csv('./data/processed/frequency_output.csv')
freq_pred.columns




Index(['policy_id', 'claim_id', 'exposure', 'claim_count', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'policy_year',
       'predicted_frequency'],
      dtype='object')

In [41]:
sev_pred = pd.read_csv('./data/processed/severity_output.csv') 
sev_pred.columns


Index(['policy_id', 'claim_id', 'accident_date', 'claim_amount', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'predicted_severity'],
      dtype='object')

In [42]:
policy = policy.merge(freq_pred[['policy_id', 'predicted_frequency']], on='policy_id')
policy = policy.merge(sev_pred[['policy_id', 'predicted_severity']], on='policy_id')  

In [43]:
policy.columns

Index(['policy_id', 'policy_start_date', 'policy_end_date', 'vehicle_age',
       'region', 'channel', 'sum_insured', 'premium', 'exposure',
       'earned_premium', 'predicted_frequency', 'predicted_severity'],
      dtype='object')

In [44]:
# Select relevant columns for final pricing output
pricing = policy[['policy_id','vehicle_age','region','channel','sum_insured','predicted_frequency','predicted_severity']]

In [46]:
#loading addesd columns and calculating final premium
# 1.40 = expense + profit + risk margin(.25+.10+.05)
pricing["pure_premium"] = pricing["predicted_frequency"] * pricing["predicted_severity"]
expense = 0.25
profit = 0.10
risk_margin = 0.05

loading_factor = 1 + expense + profit + risk_margin
pricing["final_premium"] = pricing["pure_premium"] * loading_factor
pricing.columns


C:\Windows\Temp\ipykernel_21644\3832690394.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pricing["pure_premium"] = pricing["predicted_frequency"] * pricing["predicted_severity"]
C:\Windows\Temp\ipykernel_21644\3832690394.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pricing["final_premium"] = pricing["pure_premium"] * loading_factor


Index(['policy_id', 'vehicle_age', 'region', 'channel', 'sum_insured',
       'predicted_frequency', 'predicted_severity', 'pure_premium',
       'final_premium'],
      dtype='object')

In [47]:
pricing.to_csv('./data/processed/pricing_output.csv', index=False)